In [2]:
import os

import numpy as np
from tqdm import tqdm
import cv2
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Reshape, Dense, Dropout, Flatten, LeakyReLU, Conv2D, UpSampling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import initializers

In [2]:
K.set_image_dim_ordering('tf')

latent_dim = 100
files = os.listdir('input_images')
X_train = []
for i in tqdm(files):
    x = cv2.imread(os.path.join('input_images', i))
    x = cv2.resize(x, (80, 80))
    X_train.append(x)
X_train = np.array(X_train)/255

100%|██████████| 3315/3315 [00:01<00:00, 2603.40it/s]


In [3]:
K.clear_session()
optimizer = Adam(lr=0.0002, beta_1=0.5)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
generator = Sequential()
generator.add(Dense(64*7*7, input_dim=latent_dim))
generator.add(LeakyReLU(0.2))
generator.add(Reshape((7, 7, 64)))
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(64, kernel_size=(3, 3), padding='same'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(64, kernel_size=(5, 5), padding='valid'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(64, kernel_size=(5, 5), padding='valid'))
generator.add(LeakyReLU(0.2))
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(64, kernel_size=(5, 5), padding='valid'))
generator.add(LeakyReLU(0.2))
generator.add(Conv2D(3, kernel_size=(5, 5), padding='valid', activation='sigmoid'))
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

In [5]:
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', 
                         input_shape=(80, 80, 3), kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [6]:
discriminator.trainable = False
gan_input = Input(shape=(latent_dim,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(inputs=gan_input, outputs=gan_output)
gan.compile(loss='binary_crossentropy', optimizer=optimizer)

In [11]:
def plot_images(epoch, examples=16, dim=(4, 4), figsize=(4, 4), offset=0):
    noise = np.random.normal(0, 1, size=[examples, latent_dim])
    generatedImages = generator.predict(noise)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow((generatedImages[i]*255).astype(int), interpolation='nearest')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('images/image_epoch_%d.png' % (epoch + offset))
    plt.close()

def save_models(epoch, offset=0):
    generator.save('models/epoch_%d.h5' % (epoch + offset))
    discriminator.save('models/epoch_%d.h5' % (epoch + offset))

def plot_loss(epoch, offset=0):
    plt.figure(figsize=(8, 6))
    plt.plot(g_loss, label='Generator loss')
    plt.plot(d_loss, label='Discriminator loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('images/loss_epoch_%d.png' % (epoch+offset))
    plt.close()

## Train our GAN and Plot the Synthetic Image Outputs 

After each consecutive Epoch we can see how synthetic images being improved 

In [14]:
plot_images(1)
plot_loss(1)
save_models(1)

In [ ]:
# epochs = 100
batch_size = 32
batch_count = X_train.shape[0] // batch_size
d_loss = []
g_loss = []
print('Epochs:', epochs)
print('Batch size:', batch_size)
print('Batches per epoch:', batch_count)

for e in range(1, epochs+1):
#     print('Epoch %d' % e)
    for i in tqdm(range(batch_count)):
        noise = np.random.normal(0, 1, size=[batch_size, latent_dim])
        real = X_train[np.random.randint(0, X_train.shape[0], size=batch_size)]

        generated_images = generator.predict(noise)
        X = np.concatenate([real, generated_images])

        y_dis = np.zeros(2*batch_size)
        y_dis[:batch_size] = 0.9

        discriminator.trainable = True
        dloss = discriminator.train_on_batch(X, y_dis)

        noise = np.random.normal(0, 1, size=[batch_size, latent_dim])
        y_gen = np.ones(batch_size)
        discriminator.trainable = False
        gloss = gan.train_on_batch(noise, y_gen)
#         print('.', end='')
#     print()
    
    d_loss.append(dloss)
    g_loss.append(gloss)

    plot_images(e, offset=63)
    plot_loss(e, offset=63)
    if epochs % 5 == 0:
        save_models(e, offset=63)

